In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Load the Book Data
data = pd.read_csv("Book2.csv")

In [ ]:
#Take User Input
book_title = input("Enter the name of a book: ")
book_genre = input("Enter the genre of the book: ")

In [ ]:
#Check and Drop Nulls
data = data.dropna(subset=['book_title', 'genres'])

In [ ]:
# Filter Data by Genre
genre_data = data[data['genres'] == book_genre]

In [ ]:
# Create User and Item Encoders
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

genre_data['user_id'] = user_encoder.fit_transform(genre_data['book_title'])
genre_data['item_id'] = item_encoder.fit_transform(genre_data['genres'])


In [ ]:
#Preprocessing Text
if genre_data.empty:
    print("No books found for the given genre.")
else:
    tfidf = TfidfVectorizer(stop_words='english')
    book_desc_matrix = tfidf.fit_transform(genre_data['book_desc'].values.astype('U'))
    

In [ ]:
#Create User-Item Matrix
matrix = csr_matrix((genre_data['book_rating'], (genre_data['user_id'], genre_data['item_id'])))

In [ ]:
#Perform SVD
svd = TruncatedSVD(n_components=100, random_state=42)
matrix_svd = svd.fit_transform(book_desc_matrix)

In [ ]:
#Calculate Similarity
book_idx = item_encoder.transform([book_genre])
book_vector = matrix_svd[book_idx]

similarity_scores = matrix_svd.dot(book_vector.T).flatten()

In [ ]:
#Get Top Similar Books
valid_indices = np.isin(similarity_scores.argsort()[::-1][:5], range(len(genre_data)))
similar_books_idx = similarity_scores.argsort()[::-1][:5][valid_indices]

# Ensure the label encoding classes are consistent with the dataset's range
item_encoder.classes_ = np.arange(len(genre_data))

similar_books = item_encoder.inverse_transform(similar_books_idx)

In [ ]:
#Map indices to book titles
recommended_books = [genre_data.iloc[i] for i in similar_books_idx[:4]]

# Display the recommended books
print("Your Recommended Books Are:")
for book in recommended_books:
    print(book)